# Deep Learning 1

In dit practicum gaan we het [Keras](https://keras.io/) deep learning framework voor Python gebruiken. De documentatie omschrijft Keras als volgt:

>Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

Het ontwikkelen en testen van neurale netwerken kan een vermoeiend proces zijn. Keras levert een intuïtieve API voor het trainen, testen en deployen van neurale netwerken zonder dat we ons moeten bekommeren om technische details. Om te beginnen, importeren we Keras:

In [ ]:
import keras

Keras kan gebruikt worden met verschillende *backends*. Een backend is een ander framework dat de low-level details van de implementatie afhandelt, zoals GPU optimalisatie en gedistribueerde berekeningen. Op dit moment ondersteunt Keras volgende backends: [TensorFlow](https://www.tensorflow.org/), [Theano](http://deeplearning.net/software/theano/) en [CNTK](https://www.microsoft.com/en-us/cognitive-toolkit/). Wanneer we Keras importeren, laat het ons weten welk backend het gebruikt.

## Deep Neural Network basics

Een diep neuraal network (DNN) bestaat uit verschillende lagen:

$$
    f(x) = g_L(W_Lg_{L-1}(\dots g_1(W_1x + b_1) \dots) + b_L)
$$

Hier zijn $W_1, \dots, W_L$ matrices (de *gewichten* van het netwerk), $b_1, \dots, b_L$ zijn vectoren (de *biases*) en $g_1, \dots, g_L$ zijn de *activatiefuncties*. We kunnen er een tekening bij maken:

![Een neuraal netwerk](images/mlp.png)

Elke top in dit netwerk stelt een inwendig product voor van de gewichtsvector met de input, plus een bias term. Dit resultaat gaat dan door een niet-lineaire activatiefunctie. Voorbeelden van typische activatiefuncties zijn

1. de logistic sigmoid:
$$
    \mathrm{sigmoid}(z) = \frac{1}{1 + \exp(-z)},
$$

2. de rectified linear unit (RELU):
$$
    \mathrm{relu}(z) = \max(0,z),
$$

3. de scaled exponential linear unit (SELU):
$$
    \mathrm{selu}(z) = \lambda\left\{\begin{matrix}
        z, & \mbox{als $z > 0$}\\
        \alpha(\exp(z)-1), & \mbox{als $z \leq 0$}
    \end{matrix}\right..
$$
Hier zijn $\lambda$ en $\alpha$ hyperparameters die vastliggen voordat het netwerk wordt getraind.

4. de softmax:
$$
    \mathrm{softmax}(z) = \frac{\exp(z)}{\sum_i\exp(z_i)}.
$$
De softmax wordt doorgaans gebruikt in de laatste laag van het netwerk. Het heeft de eigenschap dat voor elke $z \in \mathbb{R}^q$,
$$\begin{aligned}
    \mathrm{softmax}(z) &\in [0,1]^q, & \sum_{i=1}^q\mathrm{softmax}(z)_i &= 1.
\end{aligned}$$
Met andere woorden, de output van $\mathrm{softmax}(z)$ kan men interpreteren als een kansendistributie over $q$ mogelijkheden.

We maken nu een simpel neuraal netwerk:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(units=64, activation='relu', input_dim=30),
    Dense(units=10, activation='relu'),
    Dense(units=2, activation='softmax')
])

Dit model bestaat uit drie *dense* of *fully-connected* lagen. Dit zijn de "klassieke" lagen die gewoon een lineaire transformatie uitvoeren gevolgd door een activatiefunctie. In ons geval wordt dit netwerk formeel gegeven door

$$
    f: \mathbb{R}^{30} \to \mathbb{R}^{2}: x \mapsto \mathrm{softmax}(W_3\mathrm{relu}(W_2\mathrm{relu}(W_1x + b_1) + b_2) + b_3).
$$

Verder geldt $W_1 \in \mathbb{R}^{64 \times 30}$, $W_2 \in \mathbb{R}^{10 \times 64}$, $W_3 \in \mathbb{R}^{2 \times 10}$, $b_1 \in \mathbb{R}^{64}$, $b_2 \in \mathbb{R}^{10}$ en $b_3 \in \mathbb{R}^2$. Het aantal parameters van dit model is dus

$$
    64 \times 30 + 10 \times 64 + 2 \times 10 + 64 + 10 + 2 = 2656.
$$

Op dit moment hebben deze parameters natuurlijk nog geen nuttige waarden. Het zoeken naar waarden voor de gewichten en biases zodanig dat het netwerk een bepaalde prestatiemaat maximaliseert is het doel van een *leeralgoritme*.

## Supervised learning

In de supervised learning setting krijgen we een dataset van gelabelde observaties,

$$
    D = \{ (x_i,y_i) \in \mathbb{R}^d \times \{1, \dots, C\} \mid i = 1, \dots, N \}.
$$

We veronderstellen dat deze observaties onafhankelijk zijn en gelijk verdeeld volgens een (onbekende) kansmaat $P$. Elk element $(x_i,y_i) \in D$ bestaat uit een vector $x_i \in \mathbb{R}^d$ en een label $y_i \in \{1, \dots, C\}$. Wij willen nu een netwerk bouwen zodat

$$
    \Pr_{(x,y) \sim P}[f(x) = y] = 1.
$$

We kennen $P$ natuurlijk niet, anders zou er geen probleem zijn. We moeten dit dus indirect optimaliseren via een *empirische risicofunctie*

$$
    R_{\mathrm{emp}}(f) = \frac{1}{N}\sum_{i=1}^N\ell(f(x_i), y_i).
$$

Dit is een empirische schatting van het *verwacht risico*

$$
    R(f) = \underset{(x,y) \sim P}{\mathbb{E}}[\ell(f(x),y)],
$$

dat we niet exact kunnen bepalen. Hier is $\ell(f(x),y)$ de *verliesfunctie* die meet hoe ernstig de uitvoer van ons netwerk afwijkt van de gewenste uitvoer. Voor classificatieproblemen kan men de 0/1 loss gebruiken,

$$
    \ell(f(x),y) = \left\{\begin{matrix}
        1, & \mbox{als $f(x) \neq y$}\\
        0, & \mbox{als $f(x) = y$}
    \end{matrix}\right..
$$

Voor andere problemen zoals regressie kan het zinvoller zijn om een verlies te gebruiken zoals

$$
    \ell(f(x),y) = \|f(x)-y\|_2^2.
$$

We lossen nu volgend probleem op:

$$
    \min_f R_{\mathrm{emp}}(f).
$$

In de praktijk komt het oplossen van dit probleem neer op onze gewichten en biases aanpassen totdat we geen reductie meer krijgen van het empirisch risico. Er bestaan veel optimalisatie-algoritmen die dit probleem kunnen oplossen, maar we gaan hier niet in op details (geïnteresseerde studenten kunnen altijd het vak *Optimalisatietechnieken* volgen). Het belangrijkste is dat deze algoritmen altijd iteratief werken op *mini-batches* van de trainingdata, nooit op de volledige dataset tenzij die heel klein is. Het aantal minibatches wordt bepaald door de *batchgrootte*. Op een hoog niveau verloopt de optimalisatie dus als volgt:

1. Splits de trainingdata op in mini-batches met aantal samples gelijk aan de batchgrootte.
2. Voor elke minibatch, bereken updates aan de gewichten en biases die uitsluitend op de minibatch gebaseerd zijn.
3. Herhaal stap 2 tot convergentie.

Elke uitvoering van de lus in stap 2 noemt men een *epoch*. Je zal altijd het aantal epochs en de batchgrootte moeten opgeven als je een neuraal netwerk traint. Zorg er ook voor dat de batchgrootte een gehele deler is van de grootte van de dataset, anders zullen sommige algoritmen gewoon niet willen uitvoeren. Typische batchgroottes zijn kleine machten van 2 zoals 64 of 128. Mocht de grootte van de dataset priem zijn, zal je moeten subsamplen.

Om ons model te trainen in Keras, compileren we het met een verliesfunctie en een optimalisatie-algoritme dat dit verlies zal trachten te minimaliseren:

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Merk op dat het `metrics` argument een lijst nodig heeft, omdat we meerdere metrieken kunnen berekenen per model. Zie [de documentatie](https://keras.io/metrics/) voor een volledige lijst van ondersteunde metrieken. We kunnen het model nu fitten op een dataset, zoals de Wisconsin breast cancer data:

In [ ]:
import sklearn
import numpy as np
from sklearn.datasets import load_breast_cancer

wisconsin = load_breast_cancer()
x_data = wisconsin['data']
y_data = wisconsin['target']

We bekijken eerst wat statistieken over deze dataset:

In [ ]:
print('Aantal samples: {}'.format(x_data.shape[0]))
print('Aantal features: {}'.format(x_data.shape[1]))

class0 = x_data[y_data == 0].shape[0]
class1 = x_data[y_data == 1].shape[0]
ratio = max([class0, class1]) / min([class0, class1])
print('Klasse 0: {}'.format(class0))
print('Klasse 1: {}'.format(class1))
print('Imbalance ratio: {}'.format(ratio))

De klassen bevatten respectievelijk 212 (borstkanker) en 357 (geen borstkanker) samples. De meerderheidsklasse is dus ongeveer 1.68x zo groot als de minderheidsklasse; we gaan hier later rekening mee moeten houden. Het netwerk verwacht dat de labels vectoren zijn van dimensie 2, dus voeren we *one-hot encoding* uit:

In [ ]:
y_data = np.array([[1., 0.] if y == 0 else [0., 1.] for y in y_data])

Nu shufflen en splitsen we de data in training en test sets:

In [ ]:
from sklearn.utils import shuffle

x_data, y_data = shuffle(x_data, y_data)

p = .8
idx = int(x_data.shape[0] * p)
x_train, y_train = x_data[:idx], y_data[:idx]
x_test, y_test = x_data[idx:], y_data[idx:]

x_mean, x_std = x_train.mean(), x_train.std()
x_train -= x_mean
x_train /= x_std

x_test -= x_mean
x_test /= x_std

Merk op dat we de datasets genormaliseerd hebben zodat het gemiddelde 0 is en de variantie 1. Keras kan de parameters nu proberen schatten:

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=65)

Omwille van willekeur in de initialisatie van de parameters alsook het algoritme zelf, kan het zijn dat verschillende uitvoeringen van de `fit` methode andere resultaten geven. Je zou een accuracy van minstens 90% moeten bereiken na een handvol uitvoeringen.

Nu het model getraind is, kunnen we er voorspellingen mee doen:

In [ ]:
classes = np.argmax(model.predict(x_test, batch_size=65), axis=1)

Met deze array kunnen we de accuracy zelf berekenen:

In [ ]:
accuracy = np.mean(np.equal(classes, np.argmax(y_test, axis=1)))
print('Accuracy: {}'.format(accuracy))

Keras kan dit ook voor ons doen:

In [ ]:
evals = model.evaluate(x_test, y_test, batch_size=65)
print('Loss: {}'.format(evals[0]))
print('Accuracy: {}'.format(evals[1]))

De klassen van deze dataset zijn niet gebalanceerd, dus we zouden ook best naar de balanced accuracy kijken:

In [ ]:
labels = np.argmax(y_test, axis=1)
idx0 = (labels == 0)
idx1 = (labels == 1)

acc0 = np.mean(np.equal(classes[idx0], labels[idx0]))
acc1 = np.mean(np.equal(classes[idx1], labels[idx1]))
bal_acc = (acc0 + acc1) / 2
print('Balanced accuracy: {}'.format(bal_acc))
print('\tKlasse 0: {}'.format(acc0))
print('\tKlasse 1: {}'.format(acc1))

Normaal gezien zou er een merkbaar verschil moeten zijn tussen klasse 0 (de minderheid) en klasse 1 (de meerderheid).

## Oefening 1

Wat gebeurt er als je geen normalisatie van de data doet voordat je het netwerk traint? Kan je dit verklaren? Waarom doen we de moeite van de normalisatie apart uit te voeren op de training en test data en enkel met de statistieken berekend op de training data?

## Oefening 2

Keras ondersteunt een heel aantal verliesfuncties (zie [de documentatie](https://keras.io/losses/)). Welke zou je kiezen voor welke taak en waarom? Waarom staat de 0/1 loss die hierboven werd beschreven zelfs niet eens in deze lijst?

## Oefening 3

Keras heeft ook een aantal activatiefuncties naast de RELU (zie [de documentatie](https://keras.io/activations/)). Experimenteer met verschillende keuzes op de Wisconsin dataset. Probeer te verklaren wat je observeert. **Hint:** visualiseer de activatiefuncties eens met een plot.

## Oefening 4

Waarom maakt het uit dat de klassen niet gebalanceerd zijn? Kan je technieken bedenken om de klassen gebalanceerd te maken?

## Oefening 5

Keras heeft een aantal [ingebouwde datasets](https://keras.io/datasets/) waar je mee kunt experimenteren. Probeer eens een neuraal netwerk te trainen dat goed presteert op de Boston housing price regression data set. Merk op dat dit een *regressieprobleem* is, geen *classificatieprobleem* zoals we hiervoor beschouwd hebben. Je zal dus een lichtjes andere aanpak nodig hebben.